In [1]:
import pandas as pd
import numpy as np

In [2]:
df_1 = pd.read_csv('./data/sample_submission.csv')
df_1.head()

,ID,Adoption,Died,Euthanasia,Return_to_owner,Transfer
0,1,1,0,0,0,0
1,2,1,0,0,0,0
2,3,1,0,0,0,0
3,4,1,0,0,0,0
4,5,1,0,0,0,0


In [3]:
df_1.columns

Index(['ID', 'Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'], dtype='object')

In [4]:
df_1.sum().index

Index(['ID', 'Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'], dtype='object')

In [5]:
df_test = pd.read_csv('./data/test.csv')
df_test.sample(5)

,ID,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
1633,1634,Milo,2015-08-20 07:41:00,Cat,Neutered Male,4 months,Domestic Shorthair Mix,Black/White
7646,7647,Willow,2014-07-02 14:41:00,Cat,Spayed Female,2 years,Domestic Medium Hair Mix,Orange Tabby/White
4921,4922,Boomer,2014-08-30 11:40:00,Dog,Neutered Male,2 years,German Shorthair Pointer Mix,Red Tick
10962,10963,NaN,2014-07-04 18:42:00,Dog,Intact Female,4 months,Labrador Retriever/Australian Kelpie,Brown
7353,7354,Millard,2013-12-07 16:46:00,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Brown Tabby


In [6]:
df_test.shape

(11456, 8)

In [7]:
df_train = pd.read_csv('./data/train.csv')
df_train.sample(5)

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
20397,A699918,Bailey,2015-04-11 11:35:00,Adoption,NaN,Dog,Spayed Female,2 months,Rottweiler Mix,Black/Brown
11315,A701245,Katie,2015-04-27 19:15:00,Return_to_owner,NaN,Dog,Spayed Female,2 years,Labrador Retriever,Yellow
2098,A719336,Niiv,2016-01-25 16:13:00,Adoption,NaN,Dog,Neutered Male,1 year,Belgian Malinois Mix,Tan
12483,A709577,Mr. Pink,2015-08-28 15:46:00,Adoption,NaN,Dog,Neutered Male,5 months,Pit Bull Mix,Red/White
12854,A691754,Lolita,2014-12-06 15:49:00,Transfer,Partner,Dog,Spayed Female,3 months,Pit Bull Mix,White/Black


In [8]:
df_train.shape

(26729, 10)

# Search duplicates in train dataset

In [9]:
df_train.shape

(26729, 10)

In [10]:
# find current duplicates in train
df_train.duplicated().sum()

np.int64(0)

In [11]:
# find current duplicates in test
df_test.duplicated().sum()

np.int64(0)

In [12]:
df_train.columns

Index(['AnimalID', 'Name', 'DateTime', 'OutcomeType', 'OutcomeSubtype',
       'AnimalType', 'SexuponOutcome', 'AgeuponOutcome', 'Breed', 'Color'],
      dtype='object')

In [13]:
df_test.columns

Index(['ID', 'Name', 'DateTime', 'AnimalType', 'SexuponOutcome',
       'AgeuponOutcome', 'Breed', 'Color'],
      dtype='object')

In [14]:
# drop target columns from train.csv so we can find duplicated rows
df_train_2 = df_train.drop(['OutcomeType', 'OutcomeSubtype'], axis=1)

In [15]:
# find if column id matches between dataframes
df_test['ID'].isin(df_train['AnimalID']).sum()

np.int64(0)

In [16]:
# rename id col for matching
df_test = df_test.rename(columns={"ID": 'AnimalID'})

In [17]:
# we can now merge both dataframes so we get more data
df = pd.concat([df_train_2, df_test])
df.shape

(38185, 8)

In [18]:
# find duplicates between dataframes
df.duplicated().sum()

np.int64(0)

# 1. EDA

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38185 entries, 0 to 11455
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   AnimalID        38185 non-null  object
 1   Name            27269 non-null  object
 2   DateTime        38185 non-null  object
 3   AnimalType      38185 non-null  object
 4   SexuponOutcome  38184 non-null  object
 5   AgeuponOutcome  38161 non-null  object
 6   Breed           38185 non-null  object
 7   Color           38185 non-null  object
dtypes: object(8)
memory usage: 2.6+ MB


In [20]:
df.sample(10)

,AnimalID,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
24888,A688309,Bambi,2014-10-01 18:42:00,Dog,Intact Female,2 years,Pit Bull Mix,Fawn/White
12365,A688567,Cooper,2014-09-27 14:33:00,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Black
12367,A695642,NaN,2015-02-03 15:02:00,Cat,Spayed Female,6 months,Domestic Shorthair Mix,Black
15424,A670518,Paloma,2014-01-09 15:26:00,Dog,Intact Female,15 years,German Shepherd Mix,Brown/Black
19534,A693199,NaN,2014-12-07 15:46:00,Dog,Spayed Female,10 years,Chihuahua Shorthair Mix,Black/Tan
3122,A685078,Buddy,2014-08-07 12:49:00,Dog,Neutered Male,3 years,Chihuahua Shorthair Mix,White/Black
22041,A714883,Mr. Kitty,2015-11-24 12:17:00,Cat,Intact Male,1 year,Domestic Shorthair Mix,Brown Tabby
18531,A695335,NaN,2015-01-15 09:00:00,Cat,Unknown,1 year,Domestic Shorthair Mix,Brown Tabby
428,A690020,Jax,2014-10-15 18:25:00,Dog,Intact Female,4 months,Boxer Mix,Brown Brindle/White
1948,1949,Bijou,2013-12-22 18:45:00,Dog,Spayed Female,8 months,Miniature Pinscher Mix,Black/Tan


In [21]:
# find null values
df.isna().sum()

AnimalID              0
Name              10916
DateTime              0
AnimalType            0
SexuponOutcome        1
AgeuponOutcome       24
Breed                 0
Color                 0
dtype: int64

In [22]:
df.shape

(38185, 8)

### Before statistical analysis on numeric columns, we have to format some data (temporary)

In [23]:
# convert datetime column from object to datetime
df['DateTime'] = pd.to_datetime(df['DateTime'], errors='raise')
df['DateTime']

0       2014-02-12 18:22:00
1       2013-10-13 12:44:00
2       2015-01-31 12:28:00
3       2014-07-11 19:09:00
4       2013-11-15 12:52:00
                ...        
11451   2014-07-08 14:50:00
11452   2014-10-21 12:57:00
11453   2014-09-29 09:00:00
11454   2015-09-05 17:16:00
11455   2014-07-12 18:40:00
Name: DateTime, Length: 38185, dtype: datetime64[ns]

In [24]:
# standarize age column
df['AgeuponOutcome'].value_counts()

AgeuponOutcome
1 year       5737
2 years      5383
2 months     4929
3 years      2589
3 months     1788
1 month      1782
4 years      1506
5 years      1406
4 months     1283
3 weeks       950
5 months      929
6 years       927
6 months      835
8 years       772
7 years       769
2 weeks       727
10 months     653
10 years      598
8 months      569
4 weeks       465
7 months      455
9 years       394
12 years      330
9 months      313
1 weeks       247
11 months     241
1 week        220
13 years      214
11 years      174
3 days        148
14 years      143
2 days        139
15 years      115
1 day          84
6 days         80
4 days         78
16 years       45
5 days         38
0 years        35
17 years       30
5 weeks        15
18 years       12
20 years        7
19 years        5
22 years        2
Name: count, dtype: int64

In [25]:
def standarize_age(age):
    if pd.isna(age):
        return None
    
    value, period = age.strip().split()
    value = int(value)
    
    if 'week' in period:
        return value * 7
    elif 'month' in period:
        return value * 30
    elif 'year' in period:
        return value * 365
    else:
        return None

In [26]:
df['AgeuponOutcome'].head(10)

0      1 year
1      1 year
2     2 years
3     3 weeks
4     2 years
5     1 month
6     3 weeks
7     3 weeks
8    5 months
9      1 year
Name: AgeuponOutcome, dtype: object

In [27]:
df['AgeuponOutcome'].apply(standarize_age)

0         365.0
1         365.0
2         730.0
3          21.0
4         730.0
          ...  
11451      60.0
11452      14.0
11453     365.0
11454    2190.0
11455    1460.0
Name: AgeuponOutcome, Length: 38185, dtype: float64

In [28]:
# df_1 = df.copy()
# df = df_1.copy()

In [29]:
df['AgeuponOutcome'] = df['AgeuponOutcome'].apply(standarize_age)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38185 entries, 0 to 11455
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   AnimalID        38185 non-null  object        
 1   Name            27269 non-null  object        
 2   DateTime        38185 non-null  datetime64[ns]
 3   AnimalType      38185 non-null  object        
 4   SexuponOutcome  38184 non-null  object        
 5   AgeuponOutcome  37594 non-null  float64       
 6   Breed           38185 non-null  object        
 7   Color           38185 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 2.6+ MB


In [31]:
df['Breed'].sample(5)

3762                   Domestic Shorthair Mix
12509    Shetland Sheepdog/Chihuahua Longhair
24261                               Dachshund
9965                         Russian Blue Mix
8548                   Domestic Shorthair Mix
Name: Breed, dtype: object

In [32]:
df['Breed'].value_counts()

Breed
Domestic Shorthair Mix               12587
Pit Bull Mix                          2776
Chihuahua Shorthair Mix               2498
Labrador Retriever Mix                1949
Domestic Medium Hair Mix              1217
                                     ...  
Norfolk Terrier/Dachshund                1
Miniature Poodle/Golden Retriever        1
Labrador Retriever/Chesa Bay Retr        1
Norwegian Elkhound Mix                   1
Collie Rough/Golden Retriever            1
Name: count, Length: 1678, dtype: int64

In [33]:
df['Breed'].nunique()

1678

In [34]:
df['Color'].sample(5)

13086          Black
6878     Black/White
6094     Flame Point
7988     Black/White
8202       Tan/White
Name: Color, dtype: object

In [35]:
df['Color'].value_counts()

Color
Black/White                  4043
Black                        3265
Brown Tabby                  2359
Brown Tabby/White            1333
White                        1333
                             ... 
Brown Brindle/Brown Merle       1
Blue/Brown                      1
Calico/Orange Tabby             1
White/Flame Point               1
Gray/Blue                       1
Name: count, Length: 411, dtype: int64

In [36]:
df['Color'].nunique()

411

### Add a column isMix to reduce reed dimensionality

In [37]:
df['Breed'].str.contains('Mix')

0         True
1         True
2         True
3         True
4        False
         ...  
11451     True
11452     True
11453     True
11454     True
11455    False
Name: Breed, Length: 38185, dtype: bool

In [38]:
df.loc[df['Breed'].str.contains('Mix'), 'isMix'] = True

In [39]:
df.head(10)

,AnimalID,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,isMix
0,A671945,Hambone,2014-02-12 18:22:00,Dog,Neutered Male,365.0,Shetland Sheepdog Mix,Brown/White,True
1,A656520,Emily,2013-10-13 12:44:00,Cat,Spayed Female,365.0,Domestic Shorthair Mix,Cream Tabby,True
2,A686464,Pearce,2015-01-31 12:28:00,Dog,Neutered Male,730.0,Pit Bull Mix,Blue/White,True
3,A683430,NaN,2014-07-11 19:09:00,Cat,Intact Male,21.0,Domestic Shorthair Mix,Blue Cream,True
4,A667013,NaN,2013-11-15 12:52:00,Dog,Neutered Male,730.0,Lhasa Apso/Miniature Poodle,Tan,NaN
5,A677334,Elsa,2014-04-25 13:04:00,Dog,Intact Female,30.0,Cairn Terrier/Chihuahua Shorthair,Black/Tan,NaN
6,A699218,Jimmy,2015-03-28 13:11:00,Cat,Intact Male,21.0,Domestic Shorthair Mix,Blue Tabby,True
7,A701489,NaN,2015-04-30 17:02:00,Cat,Unknown,21.0,Domestic Shorthair Mix,Brown Tabby,True
8,A671784,Lucy,2014-02-04 17:17:00,Dog,Spayed Female,150.0,American Pit Bull Terrier Mix,Red/White,True
9,A677747,NaN,2014-05-03 07:48:00,Dog,Spayed Female,365.0,Cairn Terrier,White,NaN


In [40]:
df.loc[df['isMix'].isna(), 'isMix'] = False

In [41]:
df.head(10)

,AnimalID,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,isMix
0,A671945,Hambone,2014-02-12 18:22:00,Dog,Neutered Male,365.0,Shetland Sheepdog Mix,Brown/White,True
1,A656520,Emily,2013-10-13 12:44:00,Cat,Spayed Female,365.0,Domestic Shorthair Mix,Cream Tabby,True
2,A686464,Pearce,2015-01-31 12:28:00,Dog,Neutered Male,730.0,Pit Bull Mix,Blue/White,True
3,A683430,NaN,2014-07-11 19:09:00,Cat,Intact Male,21.0,Domestic Shorthair Mix,Blue Cream,True
4,A667013,NaN,2013-11-15 12:52:00,Dog,Neutered Male,730.0,Lhasa Apso/Miniature Poodle,Tan,False
5,A677334,Elsa,2014-04-25 13:04:00,Dog,Intact Female,30.0,Cairn Terrier/Chihuahua Shorthair,Black/Tan,False
6,A699218,Jimmy,2015-03-28 13:11:00,Cat,Intact Male,21.0,Domestic Shorthair Mix,Blue Tabby,True
7,A701489,NaN,2015-04-30 17:02:00,Cat,Unknown,21.0,Domestic Shorthair Mix,Brown Tabby,True
8,A671784,Lucy,2014-02-04 17:17:00,Dog,Spayed Female,150.0,American Pit Bull Terrier Mix,Red/White,True
9,A677747,NaN,2014-05-03 07:48:00,Dog,Spayed Female,365.0,Cairn Terrier,White,False


In [42]:
df['Breed'] = df['Breed'].str.replace('Mix', '').str.strip()

In [43]:
df['Breed'].value_counts()

Breed
Domestic Shorthair                           12781
Pit Bull                                      2877
Chihuahua Shorthair                           2623
Labrador Retriever                            2059
Domestic Medium Hair                          1284
                                             ...  
Shiba Inu/Siberian Husky                         1
Cardigan Welsh Corgi/Jack Russell Terrier        1
Norfolk Terrier/Dachshund                        1
Miniature Poodle/Golden Retriever                1
Collie Rough/Golden Retriever                    1
Name: count, Length: 1527, dtype: int64

### There are a lot of breed categories, so we will use only the first breed if there are more than 1

In [44]:
df[['Breed1', 'Breed2', 'Breed3']] = df['Breed'].str.split('/', expand=True)

In [45]:
df['Breed3'].value_counts()

Breed3
Tan Hound             9
Black Mouth Cur       2
Labrador Retriever    1
German Shepherd       1
Siberian Husky        1
Name: count, dtype: int64

In [46]:
df[['Color1', 'Color2']] = df['Color'].str.split('/', expand=True)

In [47]:
df['Color1'].value_counts()

Color1
Black                9201
White                4731
Brown Tabby          3715
Brown                2794
Tan                  2393
Orange Tabby         1820
Blue                 1742
Tricolor             1131
Red                  1107
Brown Brindle        1035
Blue Tabby            958
Tortie                829
Calico                760
Chocolate             646
Torbie                567
Sable                 459
Cream Tabby           438
Buff                  358
Yellow                350
Cream                 316
Gray                  314
Fawn                  295
Lynx Point            276
Blue Merle            240
Seal Point            234
Black Brindle         144
Flame Point           121
Gold                  103
Black Smoke           102
Black Tabby            95
Brown Merle            93
Silver                 74
Red Merle              66
Orange                 61
Gray Tabby             60
Blue Tick              58
Red Tick               56
Lilac Point            56
Silve

In [48]:
df['Color2'].value_counts()

Color2
White              12631
Black               1985
Tan                 1981
Brown               1512
Brown Brindle        231
Blue                 205
Gray                 175
Tricolor             154
Brown Tabby          149
Red                  134
Cream                111
Buff                  82
Orange Tabby          71
Chocolate             63
Orange                62
Silver                43
Blue Merle            33
Blue Tabby            27
Red Tick              24
Black Brindle         22
Blue Cream            18
Brown Merle           16
Blue Tick             14
Liver                 14
Yellow Brindle        12
Red Merle             10
Gold                  10
Yellow                 9
Black Smoke            9
Black Tabby            7
Calico                 7
Tortie                 6
Apricot                6
Cream Tabby            6
Gray Tabby             4
Lynx Point             4
Pink                   3
Chocolate Point        3
Seal Point             2
Blue Tiger        

- For practicity, we will only use the first Breed and Color columns

### Get datetime columns

In [49]:
df['hour'] = df['DateTime'].dt.hour
df['weekday'] = df['DateTime'].dt.weekday
df['month'] = df['DateTime'].dt.month

* Get TimeOfDay (morning/midday/evening/night)

In [50]:
# we'll use the cut() method
bins = [0, 5, 11, 13, 17, 21, 23]
labels = ['night', 'morning', 'midday', 'afternoon', 'evening', 'late night']

df['time_of_day'] = pd.cut(df['hour'], bins=bins, labels=labels, include_lowest=True)

* Get more derived variables

In [51]:
df.columns

Index(['AnimalID', 'Name', 'DateTime', 'AnimalType', 'SexuponOutcome',
       'AgeuponOutcome', 'Breed', 'Color', 'isMix', 'Breed1', 'Breed2',
       'Breed3', 'Color1', 'Color2', 'hour', 'weekday', 'month',
       'time_of_day'],
      dtype='object')

In [52]:
# 2 ways
# df['has_name'] = np.where(df['Name'].isna(), 'yes', 'no')
df['has_name'] = df['Name'].notna().map({True: 'yes', False: 'no'})

In [53]:
df['SexuponOutcome'].value_counts()

SexuponOutcome
Neutered Male    14014
Spayed Female    12633
Intact Female     5004
Intact Male       4985
Unknown           1548
Name: count, dtype: int64

* Separate Sex & Intact status

In [54]:
df[['status', 'sex']] = df['SexuponOutcome'].str.split(' ', expand=True)

* Fill na values with the default 'Unknown' value

In [55]:
df[['status', 'sex']] = df[['status', 'sex']].fillna('Unknown')


* LifeStage Variable (>365 days threshold)

In [56]:
df['life_stage'] = np.where(df['AgeuponOutcome'] > 365, 'adult', 'baby')

In [57]:
df.columns

Index(['AnimalID', 'Name', 'DateTime', 'AnimalType', 'SexuponOutcome',
       'AgeuponOutcome', 'Breed', 'Color', 'isMix', 'Breed1', 'Breed2',
       'Breed3', 'Color1', 'Color2', 'hour', 'weekday', 'month', 'time_of_day',
       'has_name', 'status', 'sex', 'life_stage'],
      dtype='object')

In [58]:
df['life_stage']

0         baby
1         baby
2        adult
3         baby
4        adult
         ...  
11451     baby
11452     baby
11453     baby
11454    adult
11455    adult
Name: life_stage, Length: 38185, dtype: object

In [59]:
df['life_stage'].head(10)

0     baby
1     baby
2    adult
3     baby
4    adult
5     baby
6     baby
7     baby
8     baby
9     baby
Name: life_stage, dtype: object

In [60]:
print(df['status'].value_counts())
print(df['sex'].value_counts())

status
Neutered    14014
Spayed      12633
Intact       9989
Unknown      1549
Name: count, dtype: int64
sex
Male       18999
Female     17637
Unknown     1549
Name: count, dtype: int64


In [67]:
df.isna().sum()

AnimalID              0
Name              10916
DateTime              0
AnimalType            0
SexuponOutcome        1
AgeuponOutcome      591
Breed                 0
Color                 0
isMix                 0
Breed1                0
Breed2            33828
Breed3            38171
Color1                0
Color2            18323
hour                  0
weekday               0
month                 0
time_of_day           0
has_name              0
status                0
sex                   0
life_stage            0
dtype: int64

In [69]:
df.loc[df['AgeuponOutcome'].isna()]

,AnimalID,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,isMix,Breed1,...,Color1,Color2,hour,weekday,month,time_of_day,has_name,status,sex,life_stage
179,A679441,NaN,2014-05-21 13:47:00,Cat,Intact Male,NaN,Domestic Shorthair,Cream Tabby/White,True,Domestic Shorthair,...,Cream Tabby,White,13,2,5,midday,no,Intact,Male,baby
336,A683070,NaN,2014-07-06 18:33:00,Cat,Intact Male,NaN,Domestic Shorthair,White,True,Domestic Shorthair,...,White,None,18,6,7,evening,no,Intact,Male,baby
370,A698117,NaN,2015-03-06 09:54:00,Dog,Unknown,NaN,Labrador Retriever,White,True,Labrador Retriever,...,White,None,9,4,3,morning,no,Unknown,Unknown,baby
433,A700119,NaN,2015-04-08 15:41:00,Cat,Intact Female,NaN,Domestic Shorthair,Torbie,True,Domestic Shorthair,...,Torbie,None,15,2,4,afternoon,no,Intact,Female,baby
473,A706506,NaN,2015-06-30 12:17:00,Cat,Unknown,NaN,Domestic Shorthair,Black,True,Domestic Shorthair,...,Black,None,12,1,6,midday,no,Unknown,Unknown,baby
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11028,11029,Mcpherson,2016-01-28 10:07:00,Cat,Intact Male,NaN,Domestic Longhair,Brown Tabby,True,Domestic Longhair,...,Brown Tabby,None,10,3,1,morning,yes,Intact,Male,baby
11310,11311,NaN,2015-08-31 18:09:00,Cat,Unknown,NaN,Domestic Shorthair,Blue Tabby/White,True,Domestic Shorthair,...,Blue Tabby,White,18,0,8,evening,no,Unknown,Unknown,baby
11340,11341,NaN,2014-08-29 12:08:00,Cat,Unknown,NaN,Domestic Shorthair,Blue,True,Domestic Shorthair,...,Blue,None,12,4,8,midday,no,Unknown,Unknown,baby
11343,11344,NaN,2015-08-01 16:30:00,Cat,Unknown,NaN,Siamese,Lynx Point,True,Siamese,...,Lynx Point,None,16,5,8,afternoon,no,Unknown,Unknown,baby


In [70]:
df_copy = df.copy()

In [71]:
df.columns

Index(['AnimalID', 'Name', 'DateTime', 'AnimalType', 'SexuponOutcome',
       'AgeuponOutcome', 'Breed', 'Color', 'isMix', 'Breed1', 'Breed2',
       'Breed3', 'Color1', 'Color2', 'hour', 'weekday', 'month', 'time_of_day',
       'has_name', 'status', 'sex', 'life_stage'],
      dtype='object')

In [73]:
df = df.drop(['Name', 'DateTime', 'AnimalID', 'Breed', 'Breed2', 'Breed3', 'Color', 'Color2', 'SexuponOutcome'], axis=1)

In [74]:
df.isna().sum()

AnimalType          0
AgeuponOutcome    591
isMix               0
Breed1              0
Color1              0
hour                0
weekday             0
month               0
time_of_day         0
has_name            0
status              0
sex                 0
life_stage          0
dtype: int64

- Delete missing values

In [79]:
df = df.dropna(subset=['AgeuponOutcome'])

In [80]:
df.shape

(37594, 13)

### Using Label Encoding for long categories (we'll use random forest to handle them better)

In [81]:
df.columns

Index(['AnimalType', 'AgeuponOutcome', 'isMix', 'Breed1', 'Color1', 'hour',
       'weekday', 'month', 'time_of_day', 'has_name', 'status', 'sex',
       'life_stage'],
      dtype='object')

In [84]:
df.head()

,AnimalType,AgeuponOutcome,isMix,Breed1,Color1,hour,weekday,month,time_of_day,has_name,status,sex,life_stage
0,Dog,365.0,True,Shetland Sheepdog,Brown,18,2,2,evening,yes,Neutered,Male,baby
1,Cat,365.0,True,Domestic Shorthair,Cream Tabby,12,6,10,midday,yes,Spayed,Female,baby
2,Dog,730.0,True,Pit Bull,Blue,12,5,1,midday,yes,Neutered,Male,adult
3,Cat,21.0,True,Domestic Shorthair,Blue Cream,19,4,7,evening,no,Intact,Male,baby
4,Dog,730.0,False,Lhasa Apso,Tan,12,4,11,midday,no,Neutered,Male,adult


In [ ]:
from sklearn.preprocessing import LabelEncoder

cols = ['AnimalType', 'isMix', 'Breed1', 'Color1', 'time_of_day', 'has_name', 'status', 'sex', 'life_stage']

for col in cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [ ]:
# convert from float to int
df['AgeuponOutcome'] = pd.to_numeric(df['AgeuponOutcome'], errors='coerce').astype('int')

In [104]:
df.dtypes

AnimalType        int64
AgeuponOutcome    int64
isMix             int64
Breed1            int64
Color1            int64
hour              int32
weekday           int32
month             int32
time_of_day       int64
has_name          int64
status            int64
sex               int64
life_stage        int64
dtype: object

In [108]:
# validate current values
df.shape

(37594, 13)

In [107]:
df.head()

,AnimalType,AgeuponOutcome,isMix,Breed1,Color1,hour,weekday,month,time_of_day,has_name,status,sex,life_stage
0,1,365,1,191,15,18,2,2,1,1,1,1,1
1,0,365,1,85,26,12,6,10,3,1,2,0,1
2,1,730,1,168,7,12,5,1,3,1,1,1,0
3,0,21,1,85,8,19,4,7,1,0,0,1,1
4,1,730,0,134,49,12,4,11,3,0,1,1,0


In [109]:
df_train.head()

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan


In [110]:
df_test.head()

,AnimalID,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,1,Summer,2015-10-12 12:15:00,Dog,Intact Female,10 months,Labrador Retriever Mix,Red/White
1,2,Cheyenne,2014-07-26 17:59:00,Dog,Spayed Female,2 years,German Shepherd/Siberian Husky,Black/Tan
2,3,Gus,2016-01-13 12:20:00,Cat,Neutered Male,1 year,Domestic Shorthair Mix,Brown Tabby
3,4,Pongo,2013-12-28 18:12:00,Dog,Intact Male,4 months,Collie Smooth Mix,Tricolor
4,5,Skooter,2015-09-24 17:59:00,Dog,Neutered Male,2 years,Miniature Poodle Mix,White


In [111]:
df_train.columns

Index(['AnimalID', 'Name', 'DateTime', 'OutcomeType', 'OutcomeSubtype',
       'AnimalType', 'SexuponOutcome', 'AgeuponOutcome', 'Breed', 'Color'],
      dtype='object')